In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")


In [2]:

target_column = 'Total Inflow hrly'

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

df

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS
7703,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
7702,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
7701,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
7700,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
7699,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,17,54,8,36,25,90,1,12,1,17,...,0.0,5,2,1,2,13,9,2022-06-14 11:00:00,5,25.0
3,8,62,7,43,15,105,4,16,0,20,...,0.0,4,1,3,4,13,10,2022-06-14 12:00:00,6,29.0
2,13,75,10,53,23,128,1,17,0,20,...,0.0,3,1,1,2,13,10,2022-06-14 13:00:00,2,28.0
1,12,87,8,61,20,148,2,19,0,26,...,0.0,4,0,1,2,12,9,2022-06-14 14:00:00,3,30.0


In [3]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather.ds

0       2020-01-01 00:00:00
1       2020-01-01 01:00:00
2       2020-01-01 02:00:00
3       2020-01-01 03:00:00
4       2020-01-01 04:00:00
                ...        
21501   2022-06-15 19:00:00
21502   2022-06-15 20:00:00
21503   2022-06-15 21:00:00
21504   2022-06-15 22:00:00
21505   2022-06-15 23:00:00
Name: ds, Length: 21506, dtype: datetime64[ns]

In [4]:
df = df.merge(weather, on='ds')


In [5]:
df.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
7686,17,54,8,36,25,90,1,12,1,17,...,9,2022-06-14 11:00:00,5,25.0,22.3,11.9,51.46,0.0,1.8,1013.0
7687,8,62,7,43,15,105,4,16,0,20,...,10,2022-06-14 12:00:00,6,29.0,23.9,12.0,47.03,0.0,3.6,1013.0
7688,13,75,10,53,23,128,1,17,0,20,...,10,2022-06-14 13:00:00,2,28.0,24.0,11.0,44.00,0.0,7.6,1013.0
7689,12,87,8,61,20,148,2,19,0,26,...,9,2022-06-14 14:00:00,3,30.0,23.9,12.0,47.00,0.0,5.4,1012.7
7690,8,95,13,74,21,169,3,22,0,30,...,8,2022-06-14 15:00:00,3,36.0,26.5,11.0,37.91,0.0,12.2,1013.0


In [6]:
weather_regressors = ['temp','dew','humidity','precip','windspeed','sealevelpressure']

In [7]:

m = NeuralProphet(
  # growth='off',
  yearly_seasonality=False,
  weekly_seasonality=True,
  daily_seasonality=True,
  n_lags=48,
  n_forecasts=12,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
)
for reg in weather_regressors:
  m = m.add_future_regressor(name=reg)
m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")
metrics = m.fit(df, 
                freq='H', 
                # progress='plot'
                )
print(metrics.tail(1))

with open('../models/inflow_forecast_model_with_weather.pkl', "wb") as f:
    pickle.dump(m, f)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

In [ ]:
loaded_model = pickle.load(open('../models/inflow_forecast_model_with_weather.pkl', 'rb'))


In [ ]:
last_timestamp = df.loc[len(df)-1].ds
last_timestamp

Timestamp('2022-06-12 20:00:00')

In [ ]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather[weather.ds>last_timestamp].head(12)

,ds,temp,dew,humidity,precip,windspeed,sealevelpressure
21431,2022-06-12 21:00:00,20.0,17.7,86.61,0.0,3.6,1010.0
21432,2022-06-12 22:00:00,19.5,17.3,87.12,0.0,5.4,1010.0
21433,2022-06-12 23:00:00,18.8,16.6,87.05,0.0,3.2,1010.0
21434,2022-06-13 00:00:00,18.0,16.7,92.11,1.7,5.4,1010.0
21435,2022-06-13 01:00:00,18.0,16.7,92.11,1.0,9.7,1010.0
21436,2022-06-13 02:00:00,18.0,16.4,90.37,0.0,14.4,1010.0
21437,2022-06-13 03:00:00,17.3,16.3,93.85,0.0,9.0,1010.0
21438,2022-06-13 04:00:00,17.0,16.0,93.84,0.0,10.8,1010.0
21439,2022-06-13 05:00:00,17.0,16.0,93.84,0.0,9.4,1010.0
21440,2022-06-13 06:00:00,16.8,16.0,95.03,0.0,9.0,1010.0


In [ ]:
future = pd.concat([df.tail(48),weather[weather.ds>last_timestamp].head(12)])
future.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
21438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 04:00:00,NaN,NaN,17.0,16.0,93.84,0.0,10.8,1010.0
21439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 05:00:00,NaN,NaN,17.0,16.0,93.84,0.0,9.4,1010.0
21440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 06:00:00,NaN,NaN,16.8,16.0,95.03,0.0,9.0,1010.0
21441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 07:00:00,NaN,NaN,17.8,16.3,90.94,0.0,8.3,1011.0
21442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2022-06-13 08:00:00,NaN,NaN,18.7,16.8,88.72,0.0,10.1,1011.0


In [ ]:
forecast = loaded_model.predict(future, raw=True, decompose=False)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



In [ ]:




start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/inflow_with_weather.csv', index=False)
forecast_output

,ds,inflow,timestamp
0,2022-06-12 21:00:00,11.116153,2022-06-13 00:54:18
1,2022-06-12 22:00:00,10.222641,2022-06-13 00:54:18
2,2022-06-12 23:00:00,11.255893,2022-06-13 00:54:18
3,2022-06-13 00:00:00,8.387760,2022-06-13 00:54:18
4,2022-06-13 01:00:00,5.606318,2022-06-13 00:54:18
5,2022-06-13 02:00:00,3.391297,2022-06-13 00:54:18
6,2022-06-13 03:00:00,3.133204,2022-06-13 00:54:18
7,2022-06-13 04:00:00,3.559906,2022-06-13 00:54:18
8,2022-06-13 05:00:00,3.592152,2022-06-13 00:54:18
9,2022-06-13 06:00:00,5.660214,2022-06-13 00:54:18
